In [1]:
import numpy as np
import pandas as pd
from numpy.random import randn
from numpy.linalg import inv,qr, eig ,det ,svd

import matplotlib.pyplot as plt
import random

from pandas import DataFrame , Series

## pandas读取含有重名列的数据源

In [7]:
pd.read_csv('contain_duplicate_columns.csv')#结果自动修改

,zhanghui,zhanghui.1,wudi,wuqian
0,1,53,345,45
1,2,45,45,78


In [8]:
_.columns = ['zhanghui','zhanghui','wudi','wuqian'] #但是可以赋予同名的column

In [9]:
_

,zhanghui,zhanghui,wudi,wuqian
0,1,53,345,45
1,2,45,45,78


In [10]:
_.loc[:,['zhanghui']]#这里的选取是选取了所有叫做这个名字的列，而不是只取其一

,zhanghui,zhanghui
0,1,53
1,2,45


* **上述的问题会导致传入的list和实际定位的列有差距**

In [12]:
_9.loc[:,['zhanghui','buzai']]#多出来找不到的用NaN填充一下

,zhanghui,zhanghui,buzai
0,1,53,NaN
1,2,45,NaN


### 下面进行对重复的列作为join的on字段观察

In [13]:
pd.read_csv('joined_by_contain_duplicate_columns.csv')

,zhanghui,wudi,weizhi
0,1,534,8
1,53,465,9


In [14]:
_9.merge(_13,how='inner',on='zhanghui')#报错，因为有两个同名列

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [15]:
_13.merge(_9,how='inner',on='zhanghui')#报错

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [17]:
_9.merge(_13,how='right',on='zhanghui')#报错，因为有两个同名列，left right行为也一致

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [18]:
_12

,zhanghui,zhanghui,buzai
0,1,53,NaN
1,2,45,NaN


In [19]:
_12['zhanghui'] #同样会选出两列来

,zhanghui,zhanghui
0,1,53
1,2,45


In [21]:
_12['zhanghui'] = 2

In [22]:
_12 #存在广播机制和视图非复制效应

,zhanghui,zhanghui,buzai
0,2,2,NaN
1,2,2,NaN


In [23]:
_12.columns

Index(['zhanghui', 'zhanghui', 'buzai'], dtype='object')

**特别小心选出来的columns因为重名比给定的列名多的问题，但不管怎么说，同名列应该是不应该出现的**

## merge重名的自动更改机制

In [4]:
s1 = pd.read_csv('contain_duplicate_columns.csv')#结果自动修改

In [5]:
s2 = pd.read_csv('contain_duplicate_columns.csv')

In [6]:
s1

,zhanghui,zhanghui.1,wudi,wuqian
0,1,53,345,45
1,2,45,45,78


In [8]:
s3 = s1.merge(s2,how='inner',on = 'zhanghui')
s3

,zhanghui,zhanghui.1_x,wudi _x,wuqian_x,zhanghui.1_y,wudi _y,wuqian_y
0,1,53,345,45,53,345,45
1,2,45,45,78,45,45,78


In [9]:
s4 = s1.merge(s3,how='inner',on = 'zhanghui')
s4

,zhanghui,zhanghui.1,wudi,wuqian,zhanghui.1_x,wudi _x,wuqian_x,zhanghui.1_y,wudi _y,wuqian_y
0,1,53,345,45,53,345,45,53,345,45
1,2,45,45,78,45,45,78,45,45,78


## 在merge的时候，非on字段中两个表存在重复的列（on字段名字相同）

In [14]:
s5 = s1.merge(s2,how='inner',on = 'wuqian')
s5

,zhanghui_x,zhanghui.1_x,wudi _x,wuqian,zhanghui_y,zhanghui.1_y,wudi _y
0,1,53,345,45,1,53,345
1,2,45,45,78,2,45,45


* 结果正确，同名的列区分掉了

## 在merge的时候，非on字段中两个表存在重复的列（on字段名字不相同）

In [21]:
n1 = DataFrame({'zhanghui':[1,2,3,4] , 'wudi':[17,'gx',356,23] ,'sas':[234,51,354,123]  })

In [22]:
n2 = DataFrame({'zhanghui_x':[1,2,3,5] , 'wudi':[17,23,'sd',23] ,'wudi_x':[17,23,'x356',23] ,'wudi_y':[17,23,'y356',23] ,'ddd':[234,51,354,123]  })

In [23]:
n1.merge(n2,how = 'inner',left_on = 'zhanghui' , right_on = 'zhanghui_x')

,sas,wudi_x,zhanghui,ddd,wudi_y,wudi_x,wudi_y,zhanghui_x
0,234,17,1,234,17,17,17,1
1,51,gx,2,51,23,23,23,2
2,354,356,3,354,sd,x356,y356,3


* 结果没有达到预期，出现重复的列

In [27]:
n3 = DataFrame({'zhanghui':[1,2,3,4] , 'wudi':[17,'gx',356,23] ,'sas':[234,51,354,123]  })

In [28]:
n4 = DataFrame({'zhanghui':[1,2,3,5] , 'wudi':[17,23,'sd',23] ,'wudi_x':[17,23,'x356',23] ,'wudi_y':[17,23,'y356',23] ,'ddd':[234,51,354,123]  })

In [29]:
n3.merge(n4,how = 'inner',left_on = 'zhanghui' , right_on = 'zhanghui')

,sas,wudi_x,zhanghui,ddd,wudi_y,wudi_x,wudi_y
0,234,17,1,234,17,17,17
1,51,gx,2,51,23,23,23
2,354,356,3,354,sd,x356,y356


* 只修改on字段相同，结果符合预期

## 字符串匹配

In [2]:
import re

In [3]:
re.compile('(31)|(22).*') == "214"

False

In [4]:
re.compile('(31)|(22).*') == "314"

False

In [5]:
s = re.compile('(31)|(22).*')

In [6]:
s.match('314')

<_sre.SRE_Match object; span=(0, 2), match='31'>